<a href="https://colab.research.google.com/github/Fen-ryx/Video-Super-Resolution-Project/blob/Preparatory-Code/MNIST_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%tensorflow_version 2.x

from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import pandas as pd
import numpy as np

In [3]:
from keras.datasets import mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data() # the variables defined here are not pandas DataFrames

11501568/11490434 [==============================] - 0s 0us/step


In [4]:
print('MNIST Dataset Shape:')
print('X_train: ' + str(X_train.shape))
print('Y_train: ' + str(Y_train.shape))
print('X_test:  '  + str(X_test.shape))
print('Y_test:  '  + str(Y_test.shape))

MNIST Dataset Shape:
X_train: (60000, 28, 28)
Y_train: (60000,)
X_test:  (10000, 28, 28)
Y_test:  (10000,)


In [5]:
X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)

col_name = []
for i in range(0, 784):
    col_name.append(str(i))

df_train = pd.DataFrame(X_train, columns = col_name)
df_test = pd.DataFrame(X_test, columns = col_name)
y_train = pd.DataFrame(Y_train, dtype = int)
y_test = pd.DataFrame(Y_test, dtype = int)

In [6]:
feature_columns = []
for key in df_train.keys():
    feature_columns.append(tf.feature_column.numeric_column(key = key))

print (feature_columns)

[NumericColumn(key='0', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='1', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='2', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='3', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='4', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='5', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='6', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='7', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='8', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='9', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='10', shape=(1,), default_value=None, dty

In [7]:
def input_fn(features, labels, training = True, batch_size = 256):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    if training:
        dataset = dataset.shuffle(1000).repeat()
    return dataset.batch(batch_size)

In [8]:
classifier = tf.estimator.DNNClassifier(feature_columns = feature_columns, hidden_units = [196, 14], n_classes = 10)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp73bp8uof', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [9]:
classifier.train(input_fn = lambda: input_fn(df_train, y_train), steps = 800)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp73bp8uof/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 123.10849, step = 0
INFO:tensorflow:global_step/sec: 5.41766
INFO:tensorflow:loss = 2.007866, step = 100 (18.465 sec)
INFO:tensorflow:global_step/sec: 6.00132
INFO:tensorflow:loss = 1.9837358, step = 200 (16.660 sec)
INFO:tensorflow:global_step/se

In [11]:
eval_result = classifier.evaluate(input_fn = lambda: input_fn(df_test, y_test, training = False))
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2022-05-26T02:43:40
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp73bp8uof/model.ckpt-800
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 13.21137s
INFO:tensorflow:Finished evaluation at 2022-05-26-02:43:53
INFO:tensorflow:Saving dict for global step 800: accuracy = 0.4009, average_loss = 1.741162, global_step = 800, loss = 1.7339503
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 800: /tmp/tmp73bp8uof/model.ckpt-800

Test set accuracy: 0.401

